In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn.feature_selection import RFE, f_regression
from sklearn.linear_model import (LinearRegression, Ridge, Lasso, RandomizedLasso)
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
import sklearn 
from sklearn import preprocessing 
from tpot import TPOTRegressor   
from sklearn.datasets import load_digits  
from sklearn.cross_validation import train_test_split  

C:\Users\chethan\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\chethan\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\chethan\Anaconda3\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory C:\Users\chethan\Anaconda3\lib\site-packages\mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
C:\Users\chethan\Anaconda3\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory c:\users\chethan\anaconda3\lib\site-packages\mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
C:\Users\chethan\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: Deprecatio

In [3]:
url = "https://raw.githubusercontent.com/LuisM78/Appliances-energy-prediction-data/master/energydata_complete.csv"
df = pd.read_csv(url)

In [4]:
df['date'] = pd.to_datetime(df['date'])
df['month'] , df['time'] , df['hour'] ,df['day'] , df['day_of_week'] =  df['date'].dt.month , df['date'].dt.time , df['date'].dt.hour , df['date'].dt.day , df['date'].dt.weekday_name
df['weekStatus'] = df['date'].dt.dayofweek
df['weekendstatus'] = np.where(df['weekStatus'] < 5, 0, 1)

#To Calculate NSM
df['datenew'] = df['date'].dt.date
datenum=df['datenew'].unique()
lennsm = len(datenum)+1
#print("len",lennsm)
data=[]
for j in range (1,lennsm):
    if (j==1):
        for i in range (0,42):
            if (i==0):
                a = 61200
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
    elif (j > 1 ):
        for i in range (0,144):
            if (i==0):
                a = 0
                data.append(a)
            elif (i > 0 ):
                a = a + 600
                data.append(a)
df1 = pd.DataFrame({'data_converted' : data})
dfc1 = df1[:len(df)]
df['NSM'] = dfc1
###################
df_day = pd.get_dummies(df['day_of_week'])
df = df.join(df_day)
del df['Monday']
del df['weekStatus']
del df['day_of_week']
del df['time']
del df['datenew']
#SETTING DATE COLUMN AS INDEX
df = df.set_index('date')

In [5]:
df.head(10)

,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,...,hour,day,weekendstatus,NSM,Friday,Saturday,Sunday,Thursday,Tuesday,Wednesday
date,,,,,,,,,,,,,,,,,,,,,
2016-01-11 17:00:00,60,30,19.890000,47.596667,19.20,44.790000,19.79,44.730000,19.000000,45.566667,...,17,11,0,61200,0,0,0,0,0,0
2016-01-11 17:10:00,60,30,19.890000,46.693333,19.20,44.722500,19.79,44.790000,19.000000,45.992500,...,17,11,0,61800,0,0,0,0,0,0
2016-01-11 17:20:00,50,30,19.890000,46.300000,19.20,44.626667,19.79,44.933333,18.926667,45.890000,...,17,11,0,62400,0,0,0,0,0,0
2016-01-11 17:30:00,50,40,19.890000,46.066667,19.20,44.590000,19.79,45.000000,18.890000,45.723333,...,17,11,0,63000,0,0,0,0,0,0
2016-01-11 17:40:00,60,40,19.890000,46.333333,19.20,44.530000,19.79,45.000000,18.890000,45.530000,...,17,11,0,63600,0,0,0,0,0,0
2016-01-11 17:50:00,50,40,19.890000,46.026667,19.20,44.500000,19.79,44.933333,18.890000,45.730000,...,17,11,0,64200,0,0,0,0,0,0
2016-01-11 18:00:00,60,50,19.890000,45.766667,19.20,44.500000,19.79,44.900000,18.890000,45.790000,...,18,11,0,64800,0,0,0,0,0,0
2016-01-11 18:10:00,60,50,19.856667,45.560000,19.20,44.500000,19.73,44.900000,18.890000,45.863333,...,18,11,0,65400,0,0,0,0,0,0
2016-01-11 18:20:00,60,40,19.790000,45.597500,19.20,44.433333,19.73,44.790000,18.890000,45.790000,...,18,11,0,66000,0,0,0,0,0,0


In [8]:
X = df.drop('Appliances', 1)
y = df.Appliances


In [10]:
lab_enc = preprocessing.LabelEncoder()
Y = lab_enc.fit_transform(y1)


In [14]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.5, test_size=0.5)  
  
print("X Split size")
print(len(X_train) , len(X_test))
print("Y Split size")
print(len(Y_train) , len(Y_test))

X Split size
9867 9868
Y Split size
9867 9868


In [16]:
tpot = TPOTRegressor(generations=1, verbosity=2)  
tpot.fit(X_train, Y_train)  
print(tpot.score(X_test, Y_test)) 

Generation 1 - Current best internal CV score: -61.05764752413322

Best pipeline: ExtraTreesRegressor(RobustScaler(input_matrix), bootstrap=False, max_features=0.2, min_samples_leaf=2, min_samples_split=6, n_estimators=100)
-48.24926451774624


In [17]:
tpot.export('tpot_exported_pipeline.py')

True